In [ ]:
# MY GPU IS TOO WEAK FOR THIS :(

In [1]:
import sys
print(sys.path)


['c:\\Users\\iyedm\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'c:\\Users\\iyedm\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'c:\\Users\\iyedm\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'c:\\Users\\iyedm\\AppData\\Local\\Programs\\Python\\Python310', '', 'C:\\Users\\iyedm\\AppData\\Roaming\\Python\\Python310\\site-packages', 'C:\\Users\\iyedm\\AppData\\Roaming\\Python\\Python310\\site-packages\\win32', 'C:\\Users\\iyedm\\AppData\\Roaming\\Python\\Python310\\site-packages\\win32\\lib', 'C:\\Users\\iyedm\\AppData\\Roaming\\Python\\Python310\\site-packages\\Pythonwin', 'c:\\Users\\iyedm\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages']


In [2]:
import sys
sys.path.append(r"C:\Users\iyedm\OneDrive\Desktop\NST\backend")


In [ ]:
import sys
import os

from hybrid_text.data.load_data import ProseDataset
from hybrid_text.models.hybrid_model import HybridTextModel
from hybrid_text.utils import set_seed, save_checkpoint


import torch
from torch.utils.data import DataLoader
from hybrid_text.data.load_data import ProseDataset
from hybrid_text.models.hybrid_model import HybridTextModel
from hybrid_text.utils import set_seed, save_checkpoint
from hybrid_text.data.preprocess import decoder_tokenizer   

c:\Users\iyedm\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 10
EPOCHS = 3
LR_TRANS = 3e-5
LR_GAN = 1e-4
MAX_LENGTH = 128
CHECKPOINT_DIR = "checkpoints"

set_seed(42)

train_ds = ProseDataset(split="train", max_length=MAX_LENGTH)
val_ds = ProseDataset(split="test", max_length=MAX_LENGTH)

Available columns: ['id', 'translated_dialog', 'og_response']
Available columns: ['id', 'translated_dialog', 'og_response']


In [5]:
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda b: ProseDataset.collate_fn(b, MAX_LENGTH))
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda b: ProseDataset.collate_fn(b, MAX_LENGTH))

print(f"[INFO] Training examples: {len(train_ds)}, Validation examples: {len(val_ds)}")

[INFO] Training examples: 5273, Validation examples: 3516


In [ ]:
model = HybridTextModel(device=DEVICE).to(DEVICE)

# Separate optimizer params
params_trans = list(model.encoder.model.parameters()) + list(model.decoder.model.parameters()) + list(model.decoder.proj.parameters())
params_gan = list(model.generator.parameters()) + list(model.discriminator.parameters())

optimizer_trans = torch.optim.Adam(params_trans, lr=LR_TRANS)
optimizer_gan = torch.optim.Adam(params_gan, lr=LR_GAN)

# Losses
ce_loss = torch.nn.CrossEntropyLoss(ignore_index=-100)
bce_loss = torch.nn.BCEWithLogitsLoss()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import torch
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm  # for nice progress bars

# --- Config ---
EPOCHS = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_GRAD_NORM = 1.0
ACCUM_STEPS = 4  # gradient accumulation to simulate bigger batch
LOG_EVERY = 10

# Assume model, optimizer_trans, optimizer_gan, ce_loss, bce_loss, train_loader are defined
scaler = GradScaler()  # for AMP
torch.autograd.set_detect_anomaly(True)  # optional: helps catch in-place / NaN errors

for epoch in range(1, EPOCHS + 1):
    model.train()
    epoch_ce, epoch_gan = 0.0, 0.0
    step_counter = 0

    print(f"\n=== Starting Epoch {epoch}/{EPOCHS} ===")
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}", leave=False)

    optimizer_trans.zero_grad()
    optimizer_gan.zero_grad()

    for step, batch in enumerate(progress_bar, 1):
        enc_ids = batch["enc_input_ids"].to(DEVICE)
        enc_mask = batch["enc_attention_mask"].to(DEVICE)
        dec_ids = batch["dec_input_ids"].to(DEVICE)
        dec_mask = batch["dec_attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)

        # ===========================
        # --- Forward pass ---
        # ===========================
        with autocast():
            out = model(enc_ids, enc_mask, dec_ids, dec_mask, labels=labels, train_gan=True)
            lm_logits = out["lm_logits"]
            refined = out["refined"]

        # Debug first batch only
        if step == 1:
            print("Batch shapes:", enc_ids.shape, dec_ids.shape, labels.shape)
            print("lm_logits:", lm_logits.shape, "refined:", refined.shape)

        # ---------------------------
        # Real embeddings for GAN
        # ---------------------------
        with torch.no_grad():
            _, real_repr = model.encoder(enc_ids, enc_mask)

        # ---------------------------
        # Cross-entropy loss
        # ---------------------------
        logits_trim = lm_logits[:, :labels.size(1), :].contiguous()
        loss_ce = ce_loss(logits_trim.view(-1, logits_trim.size(-1)), labels.view(-1))

        # ---------------------------
        # GAN losses
        # ---------------------------
        if torch.isnan(refined).any() or torch.isinf(refined).any():
            loss_g = torch.tensor(0.0, device=DEVICE)
            loss_d = torch.tensor(0.0, device=DEVICE)
        else:
            fake_logits = model.discriminator(refined.float())
            real_logits = model.discriminator(real_repr.detach().float())

            loss_d = 0.5 * (bce_loss(real_logits, torch.ones_like(real_logits)) +
                            bce_loss(fake_logits, torch.zeros_like(fake_logits)))
            loss_g = bce_loss(fake_logits, torch.ones_like(fake_logits))

        # ===========================
        # Backward: discriminator
        # ===========================
        if loss_d.item() > 0:
            scaler.scale(loss_d / ACCUM_STEPS).backward(retain_graph=True)

        # ===========================
        # Backward: generator + transformer
        # ===========================
        total_loss = loss_ce + 0.1 * loss_g
        if not torch.isnan(total_loss) and not torch.isinf(total_loss):
            scaler.scale(total_loss / ACCUM_STEPS).backward()
        else:
            print(f"NaN/Inf detected in total_loss at step {step}, skipping update")

        # ===========================
        # Optimizer step every ACCUM_STEPS
        # ===========================
        if step % ACCUM_STEPS == 0:
            # Clip and step discriminator
            torch.nn.utils.clip_grad_norm_(model.discriminator.parameters(), MAX_GRAD_NORM)
            scaler.step(optimizer_gan)
            scaler.update()
            optimizer_gan.zero_grad()

            # Clip and step transformer + generator
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            scaler.step(optimizer_trans)
            scaler.update()
            optimizer_trans.zero_grad()

        # ===========================
        # Logging
        # ===========================
        epoch_ce += loss_ce.item()
        epoch_gan += loss_g.item() + loss_d.item()
        step_counter += 1

        if step % LOG_EVERY == 0:
            progress_bar.set_postfix({
                "CE": f"{epoch_ce/step_counter:.4f}",
                "GAN": f"{epoch_gan/step_counter:.4f}",
                "step": step
            })

    print(f"[Epoch {epoch}] Avg CE={epoch_ce/step_counter:.4f}, Avg GAN-term={epoch_gan/step_counter:.4f}")


C:\Users\iyedm\AppData\Local\Temp\ipykernel_18260\3649950587.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # for AMP



=== Starting Epoch 1/5 ===


Epoch 1:   0%|          | 0/528 [00:00<?, ?it/s]C:\Users\iyedm\AppData\Local\Temp\ipykernel_18260\3649950587.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


--- Batch shapes ---
Encoder input_ids: torch.Size([10, 128])
Encoder attention_mask: torch.Size([10, 128])
Decoder input_ids: torch.Size([10, 128])
Decoder attention_mask: torch.Size([10, 128])
Labels: torch.Size([10, 128])
--- Token ID sanity ---
Encoder max ID: 47190 Vocab size: 50265
Decoder max ID: 250004 Vocab size: 250054
Encoder pad token: 1
Decoder pad token: 1
Batch shapes: torch.Size([10, 128]) torch.Size([10, 128]) torch.Size([10, 128])
lm_logits: torch.Size([10, 129, 250054]) refined: torch.Size([10, 768])


Epoch 1:   0%|          | 1/528 [00:08<1:17:48,  8.86s/it]

...

--- Batch shapes ---
Encoder input_ids: torch.Size([10, 128])
Encoder attention_mask: torch.Size([10, 128])
Decoder input_ids: torch.Size([10, 128])
Decoder attention_mask: torch.Size([10, 128])
Labels: torch.Size([10, 128])
--- Token ID sanity ---
Encoder max ID: 46199 Vocab size: 50265
Decoder max ID: 250004 Vocab size: 250054
Encoder pad token: 1
Decoder pad token: 1


Epoch 1:   2%|▏         | 9/528 [01:05<1:08:30,  7.92s/it]

--- Batch shapes ---
Encoder input_ids: torch.Size([10, 128])
Encoder attention_mask: torch.Size([10, 128])
Decoder input_ids: torch.Size([10, 128])
Decoder attention_mask: torch.Size([10, 128])
Labels: torch.Size([10, 128])
--- Token ID sanity ---
Encoder max ID: 44761 Vocab size: 50265
Decoder max ID: 250004 Vocab size: 250054
Encoder pad token: 1
Decoder pad token: 1


c:\Users\iyedm\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\autograd\graph.py:824: UserWarning: Error detected in NllLossBackward0. Traceback of forward call that caused the error:
  File "c:\Users\iyedm\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\iyedm\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\iyedm\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\iyedm\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\iyedm\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "C:\Users\iyedm\AppData\Roaming\Python\Python310\site-packages\tornado\pl

OutOfMemoryError: CUDA out of memory. Tried to allocate 612.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 13.46 GiB is allocated by PyTorch, and 513.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)